In [1]:
import polars as pl
import numpy as np
from typing import Tuple
import dsds.metrics as me
import dsds.prescreen as ps
import dsds.sample as sa
import dsds.fs as fs
import dsds.transform as t
import numpy as np 

In [4]:
df = pl.DataFrame({
    "time": ["2021-01-01", "2021-01-03", "2021-02-01","2021-02-11","2021-03-01","2021-03-02"],
    "a1": [None, 1,2,3,4, None,],
    "a2": [1,2, None,3,4, None,],
})
df = df.with_columns(pl.col("time").str.to_date())
ps.over_time_report(df, cols=["a1", "a2"], time_col="time", metrics=["null", "invalid", "min", "max"])

year,month,a1_null%,a2_null%,a1_invalid%,a2_invalid%,a1_min,a2_min,a1_max,a2_max
i32,u32,f64,f64,f64,f64,i64,i64,i64,i64
2021,1,0.5,0.0,0.5,0.0,1,1,1,2
2021,2,0.0,0.5,0.0,0.5,2,3,3,3
2021,3,0.5,0.5,0.5,0.5,4,4,4,4


In [ ]:
int(3.5)

In [ ]:
df = pl.scan_csv("../data/advertising.csv")
df.select(['id',
 'Daily Time Spent on Site',
 'Age',
 'Area Income',
 'Daily Internet Usage',
 'Ad Topic Line',
 'City',
 'Male',]).drop(['id',
 'Daily Time Spent on Site']).show_graph()

In [ ]:
import dsds.transform as t
df = pl.DataFrame({
    "a": [1, 1, 0],
    "b":[1, 0, 1],
    "c":[1, 1, 1]
}).lazy()

t.combine_zero_ones(df, cols=["a", "b", "c"], new_name="abc", rule="same").collect()

In [ ]:
a = np.random.random(1000)
b = np.random.random(1000)
df = pl.DataFrame({
    "a":a,
    "b":b
})
cols=["a","b"]

print(df.lazy().select(
    *(pl.col(c).quantile(0.25, interpolation="midpoint").suffix("_1") for c in cols),
    *(pl.col(c).quantile(0.75, interpolation="midpoint").suffix("_3") for c in cols),
).collect())

In [ ]:
pl.show_versions()

In [ ]:
t.scale(df, cols=["a","b"], strategy="robust")

In [ ]:
import dsds.transform as t
df = pl.DataFrame({
    "a": ['{ "id": 1, "name": "a", "course": {"0":"abc"}}', '{ "id": 2, "name": "b", "course": {"0":"efg"}}']
})

In [ ]:
df = pl.read_csv("../data/advertising.csv")
df.head()

In [ ]:
ps.describe_categories(df)

In [ ]:
import dsds.prescreen as ps
print(ps.corr_table(df, cols=["Age", "Daily Internet Usage"], corr_with=["Clicked on Ad", "Age Band"]))

In [ ]:
df2 = (
    df.lazy().select(["Area Income", "Ad Topic Line", "City", "Clicked on Ad"])
    .drop(["Ad Topic Line"])
).collect()

In [ ]:
df2.show_graph()

In [ ]:
from scipy.fft import fft
a = np.array([1,2,3,1,2,3,1,2,3])
fft(a)

In [ ]:
df = pl.read_csv("../data/train.csv").with_columns(
    pl.lit(1).alias("feature_1"),
    pl.lit(2).alias("feature_2")
)

In [ ]:
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures

In [ ]:
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()

In [ ]:
timeseries

In [ ]:
from tsfresh import extract_features
extracted_features = extract_features(timeseries, column_id="id", column_sort="time")

In [ ]:
extracted_features